In [ ]:
library(tidyverse)
library(survminer)
library(survival)

In [ ]:
clinical_data_path <- "/Users/shirin/Desktop/Immunarch/downsamplign/Projetcs/INSPIRE/clinical_data/coding_ref"
clinical_data_fname <- "INSPIRE_ClinicalData_CodingRef.xlsx"

clinical_data <- readxl::read_xlsx(
        file.path (
                clinical_data_path , clinical_data_fname))%>%
        dplyr::select(Patient_id , 
                      COHORT , 
                      `Best response longevity` ,
                      `OS (months)` ,
                      `PFS (months)`,
                      `Lost to follow up` ,
                      PFSEvent , 
                      OSEvent)

In [ ]:
FMM_data_path <- "/Users/shirin/Desktop/Immunarch/downsamplign/Projetcs/INSPIRE/Stats_Consultation/August_Revision"
FMM_data_fname <- "INSPIRE_PBMC_FMM_cluster_2023August14.xlsx"

FMM_data <- readxl::read_xlsx(
        file.path (
                FMM_data_path , FMM_data_fname))%>%
        dplyr::select(Patient_id ,  `FMM Cluster`) %>%
        rename(FMM_Cluster = `FMM Cluster`) 

In [ ]:
km_assessment <- left_join(FMM_data ,
                           clinical_data,
                           by = "Patient_id")

# PFS ASSESSMENT:

In [ ]:
pfs_km <- survminer::ggsurvplot (
        survfit (Surv(`PFS (months)`, 
                      PFSEvent ,
                      type = "right") ~ FMM_Cluster, 
                 data = km_assessment  ) ,
        ###linewidth:
        size = 0.5 ,
        #linecolor:
        palette = c(
                "#C5C8C7" , #"Baseline-High, Diminution" , 
                          "#016401" , #"Baseline-High, Stasis/Diversified" ,
                          
                          "#CD4E00" , #"Baseline-Low, Diversified" ,        
                          "#3B3029"   #"Baseline-Low, Stasis/Decline"
        ) ,
        #linetype:
        linetype = c("solid" , "solid" , "solid" , "solid") ,
        
        #Adding p-value:
        pval = FALSE ,
        pval.size = 5 ,
        pval.coord = c(0 , 0.05) ,
        
        #Fine-tuning legend features:
        legend = "right" ,
        legend.title = "Diversity shift" ,
        legend.labs = c( "Baseline-High, Diminution" , 
                         "Baseline-High, Stasis/Diversified" ,
                         "Baseline-Low, Diversified" ,  
                         "Baseline-Low, Stasis/Decline"),
        
        #Risk table:
        risk.table = TRUE,
        risk.table.col = "strata" ,
        risk.table.height = 0.25 ,
        risk.table.y.title = FALSE ) +
        ylab ("Progression-free survival\nprobability") +
        xlab ("Months")

In [ ]:
pfs_km$plot <- pfs_km$plot + 
        theme_minimal() +
        theme(
                panel.grid = element_blank() ,
                
                axis.line = element_line(color = "#000000" , linewidth = 0.1),
                axis.ticks = element_line(color = "#000000" , linewidth = 0.25) ,
                axis.ticks.length = unit(0.25 , "line") ,
                
                axis.title = element_text ( size = 15, family = "Helvetica", face="plain" , colour = "#000000"),
                
                axis.text = element_text ( size = 15, family = "Helvetica", face="plain" ,
                                           hjust = 0.5, colour = "#000000"),
                
                legend.title = element_text(family = "Helvetica", face="plain" ,
                                            color = "#000000", size = 10  ),
                legend.text  = element_text(family = "Helvetica", face="plain" ,
                                            color = "#000000", size = 10) ,
                legend.position= "none" ,
                aspect.ratio = 0.5)

In [ ]:
pfs_km$table <- pfs_km$table + 
        theme_minimal() +
        theme(panel.grid = element_blank() ,
              axis.title = element_blank(),
              axis.ticks = element_blank(),
              axis.line = element_blank(),
              
              axis.text.y = element_text ( size = 13, family = "Helvetica", face="plain" ,
                                           colour = "#000000"),
              axis.text.x = element_blank(),
              legend.position= "none" ,
              aspect.ratio = 0.13)

In [ ]:
ggsave(
        file.path(
                "/Users/shirin/Desktop/Immunarch/downsamplign/Projetcs/INSPIRE/dna/pbmc_feb/pbmc_diversity_profiles",
                "INSPIRE_PBMCShannon_FMMClusters_PFS-KM.svg" ),
        device = "svg" ,
        width = 2250,
        height = 2500,
        units = "px",
        bg = "transparent",
        dpi = 320 ,
        plot = print(pfs_km , newpage = FALSE))